# Time series predictions with Rossman Store Sales dataset

## Step 0: Necessary preprocessing / feature engineering

## Step 1: Baseline model - RFRegressor

## Step 2: More elaborate model - xgboost

## Step 3: Proper feature engineering

Sources:
- https://www.kaggle.com/elenapetrova/time-series-analysis-and-forecasts-with-prophet
- https://www.kaggle.com/danspace/rossmann-store-sales-xgboost

In [320]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [321]:
from ml_helpers.scraping import *

# 1. Load the data

## Basic info about the features
- target variable: `Sales`
- indep. variables: # of customers, was the store open, running a promo that day?, state/school holiday?

In [322]:
PATH = "data\\rossman-store-sales\\"
train_filename = open_zip(f"{PATH}train.csv.zip")
store_filename = open_zip(f"{PATH}store.csv.zip")

In [323]:
df_train = pd.read_csv(train_filename, parse_dates=True, low_memory=False)
df_store = pd.read_csv(store_filename, parse_dates=True, low_memory=False)

In [324]:
display(df_train.info()), display(df_store.info()), display(df_train.head(3))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
Store            1017209 non-null int64
DayOfWeek        1017209 non-null int64
Date             1017209 non-null object
Sales            1017209 non-null int64
Customers        1017209 non-null int64
Open             1017209 non-null int64
Promo            1017209 non-null int64
StateHoliday     1017209 non-null object
SchoolHoliday    1017209 non-null int64
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
Store                        1115 non-null int64
StoreType                    1115 non-null object
Assortment                   1115 non-null object
CompetitionDistance          1112 non-null float64
CompetitionOpenSinceMonth    761 non-null float64
CompetitionOpenSinceYear     761 non-null float64
Promo2                       1115 non-null int64
Promo2SinceWeek              571 non-null float64
Promo2SinceYear              571 non-null float64
PromoInterval                571 non-null object
dtypes: float64(5), int64(2), object(3)
memory usage: 87.2+ KB


None

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1


(None, None, None)

## Step 0: Necessary feature engineering
- the metric on the dependent variable is RMSE, therefore we need to consider the logarithm of the variable (and since some of the values are 0, `np.log1p` will be of use)
- `Date` variable needs to be encoded
- all non-numeric values need to be one-hot encoded
- additionally, there's no `Customers` variable in the test dataset, so I'm just gonna drop it from the training dataset as well

In [328]:
df_train.Sales = np.log1p(df_train.Sales)
df_train = df_train.drop(['Customers'], axis=1)
# Open_na will be formed in the test dataset, i'll manually include it here as well
df_train['Open_na'] = 0

In [332]:
df_train.columns

Index(['Store', 'DayOfWeek', 'Date', 'Sales', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday'],
      dtype='object')

## Step 1: Baseline model

I've chosen a very basic RandomForestRegressor, 5\% of the training data will serve as a validation set

Sample Performance:
- rmse for training set: 0.16758377617176337
- rmse for validation set: 0.2159886243909866
- accuracy for training set: 0.9974503664146483, 
- accuracy for validation set: 0.9953820971953687

In [333]:
from ml_helpers.manipulation import split_dataset
from ml_helpers.engineering import *
from ml_helpers.engineering import add_datepart
from ml_helpers.score import rmse

X, y = basic_preprocess_df(df_train, 'Sales', datepart='Date')

n_valid = int(len(df_train)*0.05)
n_trn = len(df_train)-n_valid
X_train, X_valid = split_dataset(X, n_trn)
y_train, y_valid = split_dataset(y, n_trn)

X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((966349, 20), (50860, 20), (966349,), (50860,))

In [334]:
from sklearn.ensemble import RandomForestRegressor
def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
          m.score(X_train, y_train), m.score(X_valid, y_valid)]
    return res

m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print(m.oob_score_)
print(print_score(m))

0.9972582812356061
[0.16758377617176337, 0.2159886243909866, 0.9974503664146483, 0.9953820971953687]


### Step 1.1: Submission using the Baseline model
- load, drop Id column, use the basic preprocessing function
- ensure that the numer of columns is the same
- sanity-check the predictions
- submit

The submission score for this solution on kaggle (30.7.2019) is 0.15466, placing me at 2526/3303-th place.‬

In [336]:
test_filename = open_zip(f"{PATH}test.csv.zip")
df_test = pd.read_csv(test_filename, parse_dates=True, low_memory=False)
ids = df_test.Id
df_test = df_test.drop(['Id'], axis=1)
X_test, _ = basic_preprocess_df(df_test, datepart='Date')

In [316]:
df_test.columns

Index(['Id', 'Store', 'DayOfWeek', 'Date', 'Open', 'Promo', 'StateHoliday',
       'SchoolHoliday'],
      dtype='object')

In [338]:
preds = m.predict(X_test)

In [340]:
np.exp(preds).mean(), np.exp(y_train).mean()

5827.144708052327

In [343]:
SUB_PATH = "data\\rossman-store-sales\\submission\\"
my_submission = pd.DataFrame({'Id': ids, 'Sales': np.exp(preds)})
my_submission.to_csv(f'{SUB_PATH}submission.csv', index=False)

Only after a very basic numericalization of continuous variables and one-hot encoding of the `Date` feature, the baseline score is pretty solid. Let's try to improve it.

In [349]:
import xgboost as xgb

def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmse(y,yhat)

params = {"objective": "reg:linear", # for linear regression
          "booster" : "gbtree",   # use tree based models 
          "eta": 0.03,   # learning rate
          "max_depth": 10,    # maximum depth of a tree
          "subsample": 0.9,    # Subsample ratio of the training instances
          "colsample_bytree": 0.7,   # Subsample ratio of columns when constructing each tree
          "silent": 1,   # silent mode
          "seed": 10   # Random number seed
          }
num_boost_round = 4000

dtrain = xgb.DMatrix(X_train, y_train)
dvalid = xgb.DMatrix(X_valid, y_valid)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
# train the xgboost model
model = xgb.train(params, dtrain, num_boost_round, evals=watchlist, \
  early_stopping_rounds= 100, feval=rmspe_xg, verbose_eval=True)

[0]	train-rmse:7.34876	eval-rmse:7.31298	train-rmspe:6967.66	eval-rmspe:6365.64
Multiple eval metrics have been passed: 'eval-rmspe' will be used for early stopping.

Will train until eval-rmspe hasn't improved in 100 rounds.
[1]	train-rmse:7.1288	eval-rmse:7.09194	train-rmspe:6967.19	eval-rmspe:6365.18
[2]	train-rmse:6.91546	eval-rmse:6.87753	train-rmspe:6966.62	eval-rmspe:6364.6
[3]	train-rmse:6.70853	eval-rmse:6.66958	train-rmspe:6965.92	eval-rmspe:6363.9
[4]	train-rmse:6.53707	eval-rmse:6.50025	train-rmspe:6965.17	eval-rmspe:6363.17
[5]	train-rmse:6.34154	eval-rmse:6.30366	train-rmspe:6964.16	eval-rmspe:6362.17
[6]	train-rmse:6.15189	eval-rmse:6.11302	train-rmspe:6962.95	eval-rmspe:6360.97
[7]	train-rmse:5.96798	eval-rmse:5.92773	train-rmspe:6961.52	eval-rmspe:6359.54
[8]	train-rmse:5.78961	eval-rmse:5.74801	train-rmspe:6959.82	eval-rmspe:6357.85
[9]	train-rmse:5.61772	eval-rmse:5.57516	train-rmspe:6957.82	eval-rmspe:6355.86
[10]	train-rmse:5.44984	eval-rmse:5.4065	train-rmspe:6955

[100]	train-rmse:0.493367	eval-rmse:0.469947	train-rmspe:3442.59	eval-rmspe:2980.06
[101]	train-rmse:0.485147	eval-rmse:0.462898	train-rmspe:3406.91	eval-rmspe:2948.36
[102]	train-rmse:0.476946	eval-rmse:0.455956	train-rmspe:3369.42	eval-rmspe:2914.59
[103]	train-rmse:0.469368	eval-rmse:0.449659	train-rmspe:3335.1	eval-rmspe:2884.06
[104]	train-rmse:0.461968	eval-rmse:0.443546	train-rmspe:3300.8	eval-rmspe:2853.56
[105]	train-rmse:0.455176	eval-rmse:0.438162	train-rmspe:3268.08	eval-rmspe:2824.54
[106]	train-rmse:0.448668	eval-rmse:0.43305	train-rmspe:3236.18	eval-rmspe:2796.28
[107]	train-rmse:0.442129	eval-rmse:0.427678	train-rmspe:3205.18	eval-rmspe:2769.26
[108]	train-rmse:0.436115	eval-rmse:0.422863	train-rmspe:3176.12	eval-rmspe:2744.33
[109]	train-rmse:0.430111	eval-rmse:0.418326	train-rmspe:3146.48	eval-rmspe:2718.79
[110]	train-rmse:0.424962	eval-rmse:0.414552	train-rmspe:3120.18	eval-rmspe:2696.21
[111]	train-rmse:0.419704	eval-rmse:0.41059	train-rmspe:3093.13	eval-rmspe:2673

[198]	train-rmse:0.302856	eval-rmse:0.346472	train-rmspe:2257.31	eval-rmspe:2058.14
[199]	train-rmse:0.302287	eval-rmse:0.346079	train-rmspe:2252.38	eval-rmspe:2054.35
[200]	train-rmse:0.30225	eval-rmse:0.346166	train-rmspe:2251.34	eval-rmspe:2054.02
[201]	train-rmse:0.301835	eval-rmse:0.345904	train-rmspe:2247.98	eval-rmspe:2051.38
[202]	train-rmse:0.30107	eval-rmse:0.345304	train-rmspe:2241.13	eval-rmspe:2045.24
[203]	train-rmse:0.300472	eval-rmse:0.344897	train-rmspe:2235.59	eval-rmspe:2040.37
[204]	train-rmse:0.299561	eval-rmse:0.344173	train-rmspe:2227.9	eval-rmspe:2033.48
[205]	train-rmse:0.299479	eval-rmse:0.344197	train-rmspe:2226.7	eval-rmspe:2032.99
[206]	train-rmse:0.29928	eval-rmse:0.344097	train-rmspe:2224.94	eval-rmspe:2031.87
[207]	train-rmse:0.299226	eval-rmse:0.344128	train-rmspe:2223.94	eval-rmspe:2031.67
[208]	train-rmse:0.298965	eval-rmse:0.343974	train-rmspe:2221.73	eval-rmspe:2030.31
[209]	train-rmse:0.298586	eval-rmse:0.343764	train-rmspe:2218.69	eval-rmspe:2027.

[296]	train-rmse:0.277252	eval-rmse:0.327525	train-rmspe:2039.87	eval-rmspe:1873.22
[297]	train-rmse:0.276978	eval-rmse:0.327274	train-rmspe:2038.12	eval-rmspe:1871.53
[298]	train-rmse:0.276891	eval-rmse:0.327185	train-rmspe:2037.42	eval-rmspe:1870.82
[299]	train-rmse:0.276842	eval-rmse:0.327145	train-rmspe:2037.02	eval-rmspe:1870.52
[300]	train-rmse:0.276433	eval-rmse:0.326799	train-rmspe:2034.38	eval-rmspe:1868.09
[301]	train-rmse:0.276431	eval-rmse:0.326871	train-rmspe:2034.29	eval-rmspe:1868.08
[302]	train-rmse:0.276118	eval-rmse:0.326591	train-rmspe:2031.86	eval-rmspe:1865.7
[303]	train-rmse:0.276113	eval-rmse:0.326593	train-rmspe:2031.78	eval-rmspe:1865.74
[304]	train-rmse:0.275804	eval-rmse:0.326313	train-rmspe:2029.39	eval-rmspe:1863.43
[305]	train-rmse:0.275535	eval-rmse:0.326094	train-rmspe:2027.75	eval-rmspe:1862
[306]	train-rmse:0.275533	eval-rmse:0.326166	train-rmspe:2027.67	eval-rmspe:1861.98
[307]	train-rmse:0.27549	eval-rmse:0.326131	train-rmspe:2027.38	eval-rmspe:1861.

[395]	train-rmse:0.256819	eval-rmse:0.309772	train-rmspe:1889.81	eval-rmspe:1731.68
[396]	train-rmse:0.256817	eval-rmse:0.309771	train-rmspe:1889.79	eval-rmspe:1731.7
[397]	train-rmse:0.256812	eval-rmse:0.309759	train-rmspe:1889.75	eval-rmspe:1731.7
[398]	train-rmse:0.256631	eval-rmse:0.309591	train-rmspe:1888.61	eval-rmspe:1730.63
[399]	train-rmse:0.256549	eval-rmse:0.309506	train-rmspe:1888.02	eval-rmspe:1730.02
[400]	train-rmse:0.256189	eval-rmse:0.309181	train-rmspe:1886.04	eval-rmspe:1727.95
[401]	train-rmse:0.255864	eval-rmse:0.308924	train-rmspe:1884.27	eval-rmspe:1726.38
[402]	train-rmse:0.255774	eval-rmse:0.308834	train-rmspe:1883.64	eval-rmspe:1725.75
[403]	train-rmse:0.255369	eval-rmse:0.308456	train-rmspe:1880.54	eval-rmspe:1722.9
[404]	train-rmse:0.255259	eval-rmse:0.308369	train-rmspe:1879.81	eval-rmspe:1722.35
[405]	train-rmse:0.254818	eval-rmse:0.307968	train-rmspe:1875.93	eval-rmspe:1718.33
[406]	train-rmse:0.254429	eval-rmse:0.307604	train-rmspe:1872.91	eval-rmspe:171

...

[3610]	train-rmse:0.158401	eval-rmse:0.221872	train-rmspe:1121.06	eval-rmspe:1052.22
[3611]	train-rmse:0.158401	eval-rmse:0.221873	train-rmspe:1121.06	eval-rmspe:1052.22
[3612]	train-rmse:0.158401	eval-rmse:0.221879	train-rmspe:1121.06	eval-rmspe:1052.22
[3613]	train-rmse:0.158401	eval-rmse:0.221882	train-rmspe:1121.06	eval-rmspe:1052.22
[3614]	train-rmse:0.158387	eval-rmse:0.221869	train-rmspe:1120.98	eval-rmspe:1052.16
[3615]	train-rmse:0.158385	eval-rmse:0.221863	train-rmspe:1120.95	eval-rmspe:1052.1
[3616]	train-rmse:0.158383	eval-rmse:0.221862	train-rmspe:1120.93	eval-rmspe:1052.1
[3617]	train-rmse:0.158383	eval-rmse:0.221862	train-rmspe:1120.93	eval-rmspe:1052.1
[3618]	train-rmse:0.158377	eval-rmse:0.221856	train-rmspe:1120.89	eval-rmspe:1052.07
[3619]	train-rmse:0.158373	eval-rmse:0.221851	train-rmspe:1120.86	eval-rmspe:1052.05
[3620]	train-rmse:0.158372	eval-rmse:0.221851	train-rmspe:1120.86	eval-rmspe:1052.06
[3621]	train-rmse:0.158372	eval-rmse:0.221859	train-rmspe:1120.86	ev

[3707]	train-rmse:0.158155	eval-rmse:0.221658	train-rmspe:1119.09	eval-rmspe:1051
[3708]	train-rmse:0.158154	eval-rmse:0.221657	train-rmspe:1119.08	eval-rmspe:1050.99
[3709]	train-rmse:0.158152	eval-rmse:0.221655	train-rmspe:1119.06	eval-rmspe:1050.99
[3710]	train-rmse:0.158152	eval-rmse:0.221652	train-rmspe:1119.06	eval-rmspe:1050.98
[3711]	train-rmse:0.158143	eval-rmse:0.221645	train-rmspe:1119	eval-rmspe:1050.93
[3712]	train-rmse:0.158135	eval-rmse:0.22164	train-rmspe:1118.94	eval-rmspe:1050.89
[3713]	train-rmse:0.158131	eval-rmse:0.221635	train-rmspe:1118.89	eval-rmspe:1050.86
[3714]	train-rmse:0.158131	eval-rmse:0.221635	train-rmspe:1118.89	eval-rmspe:1050.86
[3715]	train-rmse:0.158127	eval-rmse:0.221631	train-rmspe:1118.85	eval-rmspe:1050.84
[3716]	train-rmse:0.158117	eval-rmse:0.221624	train-rmspe:1118.81	eval-rmspe:1050.81
[3717]	train-rmse:0.158117	eval-rmse:0.22163	train-rmspe:1118.81	eval-rmspe:1050.82
[3718]	train-rmse:0.158113	eval-rmse:0.221625	train-rmspe:1118.77	eval-rm

[3804]	train-rmse:0.157881	eval-rmse:0.221419	train-rmspe:1117.14	eval-rmspe:1050.01
[3805]	train-rmse:0.157881	eval-rmse:0.221419	train-rmspe:1117.14	eval-rmspe:1050.01
[3806]	train-rmse:0.157881	eval-rmse:0.221428	train-rmspe:1117.14	eval-rmspe:1050.02
[3807]	train-rmse:0.157881	eval-rmse:0.221434	train-rmspe:1117.14	eval-rmspe:1050.02
[3808]	train-rmse:0.157878	eval-rmse:0.221433	train-rmspe:1117.11	eval-rmspe:1050.02
[3809]	train-rmse:0.157878	eval-rmse:0.221432	train-rmspe:1117.11	eval-rmspe:1050.02
[3810]	train-rmse:0.157867	eval-rmse:0.221422	train-rmspe:1117.05	eval-rmspe:1049.98
[3811]	train-rmse:0.157862	eval-rmse:0.221417	train-rmspe:1117	eval-rmspe:1049.95
[3812]	train-rmse:0.157851	eval-rmse:0.221404	train-rmspe:1116.91	eval-rmspe:1049.89
[3813]	train-rmse:0.157849	eval-rmse:0.221404	train-rmspe:1116.89	eval-rmspe:1049.92
[3814]	train-rmse:0.157845	eval-rmse:0.221402	train-rmspe:1116.87	eval-rmspe:1049.93
[3815]	train-rmse:0.157842	eval-rmse:0.221398	train-rmspe:1116.85	ev

[3901]	train-rmse:0.157564	eval-rmse:0.221159	train-rmspe:1114.88	eval-rmspe:1048.61
[3902]	train-rmse:0.157564	eval-rmse:0.22116	train-rmspe:1114.88	eval-rmspe:1048.62
[3903]	train-rmse:0.157563	eval-rmse:0.221157	train-rmspe:1114.87	eval-rmspe:1048.62
[3904]	train-rmse:0.157563	eval-rmse:0.221157	train-rmspe:1114.87	eval-rmspe:1048.62
[3905]	train-rmse:0.157561	eval-rmse:0.221157	train-rmspe:1114.85	eval-rmspe:1048.63
[3906]	train-rmse:0.157559	eval-rmse:0.221153	train-rmspe:1114.83	eval-rmspe:1048.61
[3907]	train-rmse:0.157551	eval-rmse:0.221146	train-rmspe:1114.78	eval-rmspe:1048.58
[3908]	train-rmse:0.157545	eval-rmse:0.22114	train-rmspe:1114.75	eval-rmspe:1048.53
[3909]	train-rmse:0.157537	eval-rmse:0.221126	train-rmspe:1114.69	eval-rmspe:1048.44
[3910]	train-rmse:0.157535	eval-rmse:0.221123	train-rmspe:1114.66	eval-rmspe:1048.43
[3911]	train-rmse:0.157531	eval-rmse:0.221119	train-rmspe:1114.64	eval-rmspe:1048.42
[3912]	train-rmse:0.15753	eval-rmse:0.221114	train-rmspe:1114.62	ev

[3998]	train-rmse:0.15728	eval-rmse:0.220861	train-rmspe:1112.79	eval-rmspe:1047.34
[3999]	train-rmse:0.157277	eval-rmse:0.220858	train-rmspe:1112.75	eval-rmspe:1047.29
